In [2]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('../execution_results2/execute_ils.csv')


In [3]:
df.head()

,Instance,Algorithme,Seed,Score,NbPertubations,MaxEval,CPU-Used-Time (ms)
0,instances/atsp_rand_50_75.txt,iterated_local_search_best_improver_2opt,50,1947,6,1000000,181.260
1,instances/atsp_rand_50_75.txt,iterated_local_search_best_improver_2opt,50,1971,9,1000000,172.442
2,instances/atsp_rand_50_75.txt,iterated_local_search_best_improver_2opt,51,2021,6,1000000,171.959
3,instances/atsp_rand_50_75.txt,iterated_local_search_best_improver_2opt,51,2023,9,1000000,172.661
4,instances/atsp_rand_50_75.txt,iterated_local_search_best_improver_2opt,52,1953,6,1000000,173.042


In [4]:
neighbor_relations = ["swap","2opt"]
choice_mehod = ['first', 'best']
versus = [('first', 'best')]
instances = df['Instance'].unique()
nb_perturbs = df["NbPertubations"].unique()

In [5]:
def get_score(instance,neighbor_relation,choice_mehod,seed,nbp):
    filtre = (
    (df['Instance'] == instance) &
    (df['Algorithme'] == f"iterated_local_search_{choice_mehod}_improver_{neighbor_relation}") &
    (df['Seed'] == seed) & 
    (df["NbPertubations"] == nbp)
    )
    r=df[filtre]
    if r.size != 0:
        return r.iloc[0][3]
    else:
        print(instance,neighbor_relation,choice_mehod,seed)
    

In [6]:
def gagne(instance,neighbor_relation,seed, vs,nbp):
    #print("vs:",vs)
    s1 = get_score(instance,neighbor_relation,vs[0],seed,nbp)
    s2 = get_score(instance,neighbor_relation,vs[1],seed,nbp)
    return s1 < s2

In [7]:
get_score("instances/atsp_rand_50_75.txt","2opt","best",50,9)

1971

In [8]:
get_score("instances/atsp_rand_50_75.txt","2opt","first",50,9)

1880

In [9]:
gagne("instances/atsp_rand_50_75.txt","2opt",50, ("first","best"),9)

True

In [10]:
df["Seed"].unique()

array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59])

In [11]:
result_tuple={"instances" : instances}
for instance in instances:
    #print(instance)
    for nr in neighbor_relations:
        #print(nr)
        for vs in versus:
            #print(vs," : ",end="")
            r=[0,0]
            for nbp in nb_perturbs:
                for seed in df["Seed"].unique():
                    if gagne(instance,nr,seed, vs,nbp):
                        r[0]+=1
                    else:
                        r[1]+=1
            try:
                result_tuple[f"{nr}_{vs[0]}_vs_{vs[1]}"].append((r[0],r[1]))
            except:
                result_tuple[f"{nr}_{vs[0]}_vs_{vs[1]}"] = [(r[0],r[1])]
            #print(r)
                
result_tuple = pd.DataFrame(result_tuple)
result_tuple

,instances,swap_first_vs_best,2opt_first_vs_best
0,instances/atsp_rand_50_75.txt,"(11, 9)","(15, 5)"
1,instances/atsp_rand_50_100.txt,"(5, 15)","(19, 1)"
2,instances/atsp_rand_60_50.txt,"(11, 9)","(17, 3)"
3,instances/atsp_rand_60_100.txt,"(6, 14)","(18, 2)"
4,instances/atsp_rand_70_55.txt,"(9, 11)","(18, 2)"
5,instances/atsp_rand_70_100.txt,"(13, 7)","(20, 0)"
6,instances/atsp_rand_80_70.txt,"(10, 10)","(18, 2)"
7,instances/atsp_rand_80_100.txt,"(10, 10)","(18, 2)"


In [12]:
df_group = df[["Algorithme","Score"]].groupby("Algorithme")
df_group.mean()

,Score
Algorithme,
iterated_local_search_best_improver_2opt,1654.21875
iterated_local_search_best_improver_swap,1161.33750
iterated_local_search_first_improver_2opt,1496.33125
iterated_local_search_first_improver_swap,1163.11250


In [13]:
df_group = df[["Algorithme","CPU-Used-Time (ms)"]].groupby("Algorithme")
df_group.mean()

,CPU-Used-Time (ms)
Algorithme,
iterated_local_search_best_improver_2opt,223.922163
iterated_local_search_best_improver_swap,188.638675
iterated_local_search_first_improver_2opt,229.166100
iterated_local_search_first_improver_swap,191.710950
